# Deploy personalize and ranking campaign

#### To deploy with your own data, change the following 4 parameters:
1.dataset_group_name: the name of the dataset group 

2.dataset_name:the name of the dataset 

3.interactions_schema: the schema of the interaction data 

4.ranking_campaign_name: the name of the ranking campaign 

### Import Personalize and S3

In [4]:
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizemanagedretailers"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket = bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
except s3.exceptions.BucketAlreadyOwnedByYou:
    print("Bucket already exists. Using bucket", bucket_name)

region: us-east-2
bucket_name: 513489159680-us-east-2-personalizemanagedretailers


### upload interaction data to s3

In [5]:
interactions_file = 'interaction_data.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_file).upload_file(interactions_file)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_file

### Set the S3 bucket policy

In [6]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '81EKY19V8N07PDW6',
  'HostId': 'kHppDj16C9l5zAq89t7E/mTLeEp62sjz60PcLugYN6gdxivjvJTr9O6egztuvfczDzqigQJ+5qn75Hth/3Aikg==',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'kHppDj16C9l5zAq89t7E/mTLeEp62sjz60PcLugYN6gdxivjvJTr9O6egztuvfczDzqigQJ+5qn75Hth/3Aikg==',
   'x-amz-request-id': '81EKY19V8N07PDW6',
   'date': 'Sat, 18 Nov 2023 06:45:14 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Create Dataset Group

In [8]:
dataset_group_name = 'personalize_ecomemerce_ds_group'
response = personalize.create_dataset_group(
    name=dataset_group_name,
    domain='ECOMMERCE'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-2:513489159680:dataset-group/personalize_ecomemerce_ds_group",
  "domain": "ECOMMERCE",
  "ResponseMetadata": {
    "RequestId": "f3e0f73a-0a61-4356-a3ad-e03c826feb79",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 18 Nov 2023 06:55:49 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "131",
      "connection": "keep-alive",
      "x-amzn-requestid": "f3e0f73a-0a61-4356-a3ad-e03c826feb79"
    },
    "RetryAttempts": 0
  }
}


In [9]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: ACTIVE
CPU times: user 5.39 ms, sys: 25 µs, total: 5.41 ms
Wall time: 61.8 ms


### Create Interactions Schema

In [10]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
            
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-interatn_group",
    domain = "ECOMMERCE",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-2:513489159680:schema/personalize-ecommerce-interatn_group",
  "ResponseMetadata": {
    "RequestId": "1f02b53d-bef5-4d49-b8a8-39631799ea69",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 18 Nov 2023 06:56:12 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "102",
      "connection": "keep-alive",
      "x-amzn-requestid": "1f02b53d-bef5-4d49-b8a8-39631799ea69"
    },
    "RetryAttempts": 0
  }
}


### Create Interactions Dataset

In [11]:
dataset_name = "personalize_ecommerce_demo_interactions"
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = dataset_name,
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-2:513489159680:dataset/personalize_ecomemerce_ds_group/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "22bca2d4-a8fd-47dd-85df-b7af8ae76bdb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 18 Nov 2023 06:56:20 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "112",
      "connection": "keep-alive",
      "x-amzn-requestid": "22bca2d4-a8fd-47dd-85df-b7af8ae76bdb"
    },
    "RetryAttempts": 0
  }
}


### Create Personalize Role

In [14]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleEcommerceDemoRecommender"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# # Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)


arn:aws:iam::513489159680:role/PersonalizeRoleEcommerceDemoRecommender


## Import the data

### Create Interactions Dataset Import Job

In [16]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_interactions_import",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_file)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-2:513489159680:dataset-import-job/personalize_ecommerce_demo_interactions_import",
  "ResponseMetadata": {
    "RequestId": "eec290ad-808b-4b8e-a177-b0997bbc41d3",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 18 Nov 2023 07:00:11 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "134",
      "connection": "keep-alive",
      "x-amzn-requestid": "eec290ad-808b-4b8e-a177-b0997bbc41d3"
    },
    "RetryAttempts": 0
  }
}


In [17]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 54.4 ms, sys: 6.02 ms, total: 60.5 ms
Wall time: 4min


## build ranking recipe

In [18]:
rank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"
rank_create_solution_response = personalize.create_solution(
    name = "personalize-ranking",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rank_recipe_arn
)

rank_solution_arn = rank_create_solution_response['solutionArn']
print(json.dumps(rank_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-2:513489159680:solution/personalize-ranking",
  "ResponseMetadata": {
    "RequestId": "42eb31d7-86f0-4718-9e88-21eda813b3a3",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 18 Nov 2023 07:07:56 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "89",
      "connection": "keep-alive",
      "x-amzn-requestid": "42eb31d7-86f0-4718-9e88-21eda813b3a3"
    },
    "RetryAttempts": 0
  }
}


In [19]:
rank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rank_solution_arn
)
rank_solution_version_arn = rank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rank_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-2:513489159680:solution/personalize-ranking/5d69c6fa",
  "ResponseMetadata": {
    "RequestId": "d413ebe3-3c6e-4610-a622-728098bafd20",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 18 Nov 2023 07:07:59 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "105",
      "connection": "keep-alive",
      "x-amzn-requestid": "d413ebe3-3c6e-4610-a622-728098bafd20"
    },
    "RetryAttempts": 0
  }
}


In [20]:
in_progress_solution_versions = [
    rank_solution_version_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
Build succeeded for arn:aws:personalize:us-east-2:513489159680:soluti

### build ranking campaign

In [22]:
ranking_campaign_name = "personalize-poc-rerank"
rank_create_campaign_response = personalize.create_campaign(
    name = ranking_campaign_name,
    solutionVersionArn = rank_solution_version_arn,
    minProvisionedTPS = 1
)
rank_campaign_arn = rank_create_campaign_response['campaignArn']
print(json.dumps(rank_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-2:513489159680:campaign/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "bfcd89cb-1103-4bdc-990e-dccf246986fe",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 18 Nov 2023 07:29:42 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "92",
      "connection": "keep-alive",
      "x-amzn-requestid": "bfcd89cb-1103-4bdc-990e-dccf246986fe"
    },
    "RetryAttempts": 0
  }
}


In [24]:
in_progress_campaigns = [
    rank_campaign_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-2:513489159680:campaign/personalize-poc-rerank


### Test

In [25]:
user_id = '3156'
item_id_list = ['b93b7b15-9bb3-407c-b80b-517e7c45e090','3946f4c8-1b5b-4161-b794-70b33affb671','b98a3579-2a92-47e5-a9ae-65d776c76ac3']

personalize_runtime = boto3.client('personalize-runtime')
result = personalize_runtime.get_personalized_ranking(
    campaignArn = rank_campaign_arn,
    userId = user_id,
    inputList = item_id_list
)

ranking_result = result['personalizedRanking']        
print("ranking_result:",ranking_result)

ranking_result: [{'itemId': 'b93b7b15-9bb3-407c-b80b-517e7c45e090', 'score': 0.9012777}, {'itemId': '3946f4c8-1b5b-4161-b794-70b33affb671', 'score': 0.0760749}, {'itemId': 'b98a3579-2a92-47e5-a9ae-65d776c76ac3', 'score': 0.0226474}]
